In [6]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from Levenshtein import ratio
import eng_to_ipa
import json

In [4]:
id_freq = pd.read_csv('identifier_frequency.csv')
top_identifiers = id_freq['identifier'].astype(str)

translation_dict = {}

for id in tqdm(top_identifiers):
    id_ipa = eng_to_ipa.convert(id)
    translation_dict[id] = id_ipa

100%|██████████| 81297/81297 [06:52<00:00, 197.28it/s]


In [13]:
translate = {}
translatable = 0

for k, v in translation_dict.items():
    if k in v:
        pass
    else:
        translate[k] = v
        translatable += 1

print(translatable/81297)

f = open('ipa_translation_dict.json', 'w')
f.write(json.dumps(translate))
f.close()

print(len(translate))

0.06532836390026692
5311


In [23]:
words = list(translate)


size = len(words)

result = np.zeros((size, size))

for i in tqdm(range(size)):
    for j in range(size):
        if i < j:
            result[i][j] = ratio(translate[words[i]], translate[words[j]])

sorted = np.flip(np.argsort(result.flatten()))

100%|██████████| 5311/5311 [00:07<00:00, 679.02it/s] 


In [26]:
f = open('most_similar_identifiers.csv', 'w')

for n in range(10000):
    i, j = np.unravel_index(sorted[n], (size, size))
    if words[i] not in words[j] and words[j] not in words[i]:
        f.write(words[i] + ',' + words[j] + ',' + str(result[i][j]) + '\n')

f.close()